In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
data = pd.read_csv("Police_Department_Incident_Reports__Historical_2003_to_May_2018_20250208.csv") 
from urllib.request import urlopen
import json
import folium
from folium import plugins
from folium.plugins import HeatMap


FileNotFoundError: [Errno 2] No such file or directory: 'Police_Department_Incident_Reports__Historical_2003_to_May_2018_20250208.csv'

In [ ]:
cleandata=data.drop(columns=['SF Find Neighborhoods 2 2','Current Police Districts 2 2', 'Current Supervisor Districts 2 2',
       'Analysis Neighborhoods 2 2', 'DELETE - Fire Prevention Districts 2 2',
       'DELETE - Police Districts 2 2', 'DELETE - Supervisor Districts 2 2',
       'DELETE - Zip Codes 2 2', 'DELETE - Neighborhoods 2 2',
       'DELETE - 2017 Fix It Zones 2 2',
       'Civic Center Harm Reduction Project Boundary 2 2',
       'Fix It Zones as of 2017-11-06  2 2', 'DELETE - HSOC Zones 2 2',
       'Fix It Zones as of 2018-02-07 2 2',
       'CBD, BID and GBD Boundaries as of 2017 2 2',
       'Areas of Vulnerability, 2016 2 2',
       'Central Market/Tenderloin Boundary 2 2',
       'Central Market/Tenderloin Boundary Polygon - Updated 2 2',
       'HSOC Zones as of 2018-06-05 2 2', 'OWED Public Spaces 2 2', 'Address',
       'Neighborhoods 2', 'PdId'])

cleandata['Date'] = pd.to_datetime(cleandata['Date'], format='%m/%d/%Y')

crimedata2 = pd.read_csv("Police_Department_Incident_Reports__2018_to_Present_20250208.csv") 

crimedata2 = crimedata2.replace(['Recovered Vehicle', 'Larceny Theft', 'Lost Property', 'Non-Criminal', 'Drug Violation', 'Assault', 'Malicious Mischief', 'Fraud', 'Warrant', 'Other Offenses', 'Robbery', 'Case Closure', 'Stolen Property','Suspicious Occ', 'Disorderly Conduct', 'Weapons Carrying Etc', 'Rape', 'Drug Offense', 'Missing Person', 'Motor Vehicle Theft', 'Burglary', 'Fire Report', 'Arson', 'Vandalism', 'Suicide', 'Traffic Violation Arrest', 'Forgery And Counterfeiting', 'Sex Offense', 'Prostitution', 'Weapons Offense', 'Miscellaneous Investigation', 'Vehicle Misplaced', 'Suspicious', 'Embezzlement', 'Vehicle Impounded', 'Civil Sidewalks', 'Liquor Laws', 'Gambling', 'Motor Vehicle Theft?', 'Weapons Offence', 'Homicide',  'Other Miscellaneous', 'Other', 'Human Trafficking (A), Commercial Sex Acts', 'Human Trafficking, Commercial Sex Acts', 'Human Trafficking (B), Involuntary Servitude', 'Traffic Collision', 'Courtesy Report', 'Offences Against The Family And Children'], ['RECOVERED VEHICLE','LARCENY/THEFT', 'NON-CRIMINAL', 'NON-CRIMINAL', 'DRUG/NARCOTIC', 'ASSAULT', 'VANDALISM', 'FRAUD', 'WARRANTS', 'OTHER OFFENSES', 'ROBBERY', 'NON-CRIMINAL' , 'STOLEN PROPERTY', 'SUSPICIOUS OCC', 'DISORDERLY CONDUCT', 'WEAPON LAWS', 'SEX OFFENSES, FORCIBLE', 'DRUG/NARCOTIC', 'MISSING PERSON', 'VEHICLE THEFT', 'BURGLARY', 'NON-CRIMINAL', 'ARSON', 'VANDALISM', 'SUICIDE', 'OTHER OFFENSES', 'FORGERY/COUNTERFEITING', 'SEX OFFENSES, NON FORCIBLE', 'PROSTITUTION', 'WEAPON LAWS', 'NON-CRIMINAL', 'NON-CRIMINAL', 'SUSPICIOUS OCC', 'EMBEZZLEMENT', 'SECONDARY CODES', 'DISORDERLY CONDUCT', 'LIQUOR LAWS', 'GAMBLING', 'VEHICLE THEFT', 'WEAPON LAWS', 'ASSAULT', 'SECONDARY CODES', 'SECONDARY CODES', 'SEX OFFENSES, FORCIBLE', 'SEX OFFENSES, FORCIBLE', 'KIDNAPPING', 'NON-CRIMINAL','NON-CRIMINAL', 'OTHER OFFENSES'])

crimeCategory = pd.unique(crimedata2["Incident Category"])

crimedata2cleaned=crimedata2.drop(columns=['Neighborhoods', 'ESNCAG - Boundary File',
       'Central Market/Tenderloin Boundary Polygon - Updated',
       'Civic Center Harm Reduction Project Boundary',
       'Analysis Neighborhood', 'Supervisor District',
       'Supervisor District 2012',      
       'HSOC Zones as of 2018-06-05', 'Invest In Neighborhoods (IIN) Areas',
       'Current Supervisor Districts', 'Current Police Districts', 'Report Type Code',
       'Report Type Description', 'Filed Online', 'Report Datetime', 'CAD Number', 'Incident Subcategory',
       'Intersection', 'CNN','Incident Datetime', 'Row ID', 'Incident Year', 'Incident ID'])

crimedata2cleaned = crimedata2cleaned.dropna(subset=["Incident Category"])

crimedata2cleaned['Incident Date'] = pd.to_datetime(crimedata2cleaned['Incident Date'], format='%Y/%m/%d')

crimedata2cleaned = crimedata2cleaned.iloc[:, [3, 4, 5, 6, 2, 0, 1, 8, 7, 10, 9, 11,]]    

crimedata2cleaned = crimedata2cleaned[crimedata2cleaned['Incident Date'].dt.year != 2025]

crimedata2cleaned = crimedata2cleaned.rename(columns={"Incident Number": "IncidntNum", "Incident Code": "Incident Code", "Incident Category": "Category", "Incident Description": "Descript", "Incident Day of Week": "DayOfWeek", "Incident Date": "Date", "Incident Time": "Time", "Police District": "PdDistrict", "Resolution": "Resolution", "Longitude": "X", "Latitude": "Y", "Point": "location"})

allcrime = pd.concat([cleandata, crimedata2cleaned], ignore_index=True)

focuscrimes = set(['WEAPON LAWS', 'PROSTITUTION', 'ROBBERY', 'BURGLARY', 'ASSAULT', 'DRUG/NARCOTIC', 'LARCENY/THEFT', 'VANDALISM', 'VEHICLE THEFT', 'STOLEN PROPERTY'])

allcrimeCategory = pd.unique(allcrime["Category"])
focuscrimes = np.array(list(focuscrimes))

boolstuff = ~np.isin(allcrimeCategory, focuscrimes)
catToToss = allcrimeCategory[boolstuff]
allFocuscrimes = allcrime[~allcrime['Category'].isin(catToToss)]
allFocuscrimes=allFocuscrimes.drop(columns=['Descript'])
allFocuscrimes['Time_real'] = pd.to_datetime(allFocuscrimes['Time'], format='%H:%M')
allFocuscrimes


In [ ]:
df_drugs = allFocuscrimes[allFocuscrimes["Category"] == "DRUG/NARCOTIC"]
df_drugs = df_drugs.dropna()
df_drugs

In [ ]:
lat = 37.828724
lon = -122.355537

map_drugs = folium.Map([lat, lon], zoom_start=12)
heat_df = df_drugs[['Y', 'X']]
heat_data = [[row['Y'],row['X']] for index, row in heat_df.iterrows()]

# Plot it on the map
HeatMap(heat_data).add_to(map_drugs)

# Display the map
map_drugs

In [ ]:
df_drugs["Date"] = pd.to_datetime(df_drugs["Date"])
df_drugs["year"] = df_drugs["Date"].dt.year
df_drugs

In [ ]:
lat = 37.776669
lon = -122.452708
map_drugs_time = folium.Map([lat, lon], zoom_start=12)

heat_df = df_drugs.loc[:, ['Y', 'X']]

# Create weight column, using date
heat_df.loc[:,'Weight'] = df_drugs['year']
heat_df = heat_df.dropna(axis=0, subset=['Y','X', 'Weight'])

# List comprehension to make out list of lists
heat_data = [[[row['Y'],row['X']] for index, row in heat_df[heat_df['Weight'] == i].iterrows()] for i in range(2003,2024)]

# Parameters
hm = plugins.HeatMapWithTime(
    heat_data,
    index=list(range(2003, 2024)),  # Explicitly set years as time labels
    auto_play=True,
    max_opacity=0.8,
    radius=10,
    blur=0.5,
    position="bottomright"
)
hm.add_to(map_drugs_time)


# Add heat map to the Folium map
hm.add_to(map_drugs_time)

# Display the map
map_drugs_time

In [ ]:
map_drugs_time.save("heatmapdrugs.html")